# Model Evaluation

## Four-fold Cross Evaluation

```
Fold 1: trained on FRH01, FRH02, FRH03, tested on FRH04
Fold 2: trained on FRH01, FRH02, FRH04, tested on FRH03
Fold 3: trained on FRH01, FRH03, FRH04, tested on FRH02
Fold 4: trained on FRH02, FRH03, FRH04, tested on FRH01
```

## Download and unzip evaluation runs
to `/tmp`


Folder structure
```
<level>/
    <fold>/
        <model>/
            y_pred.npy
            y_true.npy
            classification_report.txt
            model.pth
            ...
```

In [22]:
!wget -N -P /tmp https://syncandshare.lrz.de/dl/fi6zpj2xQ24mM3VxLewj87eQ/L1C.zip
!unzip -o /tmp/L1C.zip -d /tmp

!wget -N -P /tmp https://syncandshare.lrz.de/dl/fi5D238TjzBYFr78ERmeUbCd/L2A.zip    
!unzip -o /tmp/L2A.zip -d /tmp

--2020-05-02 22:15:35--  https://syncandshare.lrz.de/dl/fi6zpj2xQ24mM3VxLewj87eQ/L1C.zip
Resolving syncandshare.lrz.de (syncandshare.lrz.de)... 129.187.255.213
Connecting to syncandshare.lrz.de (syncandshare.lrz.de)|129.187.255.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 284560207 (271M) [application/x-zip-compressed]
Saving to: ‘/tmp/L1C.zip’

L1C.zip             100%[===================>] 271,38M  11,2MB/s    in 24s     

2020-05-02 22:16:00 (11,2 MB/s) - ‘/tmp/L1C.zip’ saved [284560207/284560207]

--2020-05-02 22:16:00--  https://syncandshare.lrz.de/dl/fi5D238TjzBYFr78ERmeUbCd/L2A.zip
Resolving syncandshare.lrz.de (syncandshare.lrz.de)... 129.187.255.213
Connecting to syncandshare.lrz.de (syncandshare.lrz.de)|129.187.255.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279045345 (266M) [application/x-zip-compressed]
Saving to: ‘/tmp/L2A.zip’

L2A.zip             100%[===================>] 266,12M  11,2MB/s    in 24s     

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

import sys
sys.path.append("..")
from examples.train import metrics as calc_metrics

def load(run):
    y_pred = np.load(os.path.join(run,"y_pred.npy"))
    y_true = np.load(os.path.join(run,"y_true.npy"))
    y_score = np.load(os.path.join(run,"y_score.npy"))
    field_ids = np.load(os.path.join(run,"field_ids.npy"))
    rs = pd.DataFrame([y_pred,y_true,field_ids],index=["y_pred","y_true","field_ids"]).T.set_index("field_ids")
    return rs, y_score

def load_table(logdir):
    runs = os.listdir(logdir)
    #runs = ["LSTM","OmniScaleCNN","MSResNet","StarRNN","TempCNN","TransformerEncoder", "InceptionTime"]

    stats = list()
    for run in runs:
        rs, _ = load(os.path.join(logdir,run))
        stat = calc_metrics(rs.y_true,rs.y_pred)
        stat["model"] = run
        stats.append(stat)
    stats = pd.DataFrame(stats).set_index("model")

    df = stats.T
    #df["RF"] = ""

    #models = ["RF","OmniScaleCNN", "TempCNN","MSResNet", "InceptionTime", "LSTM","StarRNN","TransformerEncoder"]
    table = df.loc[["accuracy","recall_macro","f1_macro","f1_weighted","kappa"]]
    table.index = ["overall accuracy","average accuracy","class-mean f-score","weighted f-score","kappa-metric"]
    return table

### Compile L1C Results

In [6]:
logdir = "/tmp"

l1tables = []
for fold in [1,2,3,4]:
    table = load_table(f"{logdir}/L1C/{fold}").T
    table["fold"] = fold
    l1tables.append(table)


/home/marc/miniconda3/envs/breizhcrops/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [21]:
meantable = pd.concat(l1tables).groupby(["model"]).mean()
stdtable = pd.concat(l1tables).groupby(["model"]).std()

metrics = ["overall accuracy","average accuracy","class-mean f-score","weighted f-score","kappa-metric"]
models = ["OmniScaleCNN","TempCNN","MSResNet", "InceptionTime", "LSTM", "StarRNN", "TransformerEncoder"]
for metric in metrics:
    entries = list()
        
    for model in models:
        entries.append("$" + f"{meantable.loc[model,metric]:.2f}"+"^{\\pm "+ f"{stdtable.loc[model,metric]:.2f}" + "}$")
    row = f"{metric} & & " + " & ".join(entries)
    row += " \\\ "
    print(row)

overall accuracy & & $0.78^{\pm 0.02}$ & $0.78^{\pm 0.03}$ & $0.70^{\pm 0.06}$ & $0.73^{\pm 0.06}$ & $0.79^{\pm 0.03}$ & $0.79^{\pm 0.02}$ & $0.80^{\pm 0.01}$ \\ 
average accuracy & & $0.54^{\pm 0.02}$ & $0.55^{\pm 0.01}$ & $0.54^{\pm 0.05}$ & $0.53^{\pm 0.04}$ & $0.56^{\pm 0.02}$ & $0.56^{\pm 0.01}$ & $0.57^{\pm 0.01}$ \\ 
class-mean f-score & & $0.55^{\pm 0.02}$ & $0.54^{\pm 0.03}$ & $0.55^{\pm 0.04}$ & $0.50^{\pm 0.04}$ & $0.56^{\pm 0.03}$ & $0.56^{\pm 0.02}$ & $0.58^{\pm 0.01}$ \\ 
weighted f-score & & $0.77^{\pm 0.02}$ & $0.77^{\pm 0.04}$ & $0.69^{\pm 0.06}$ & $0.71^{\pm 0.07}$ & $0.77^{\pm 0.04}$ & $0.76^{\pm 0.04}$ & $0.80^{\pm 0.01}$ \\ 
kappa-metric & & $0.71^{\pm 0.02}$ & $0.72^{\pm 0.04}$ & $0.63^{\pm 0.07}$ & $0.64^{\pm 0.08}$ & $0.72^{\pm 0.05}$ & $0.72^{\pm 0.03}$ & $0.74^{\pm 0.02}$ \\ 


In [7]:
pd.concat(l1tables).groupby(["model","fold"]).first()

overall accuracy  average accuracy  \
model              fold                                       
InceptionTime      1             0.790000          0.561291   
                   2             0.760876          0.536852   
                   3             0.660557          0.477702   
                   4             0.691921          0.526863   
LSTM               1             0.803098          0.573440   
                   2             0.779778          0.556539   
                   3             0.743240          0.523667   
                   4             0.820651          0.577291   
MSResNet           1             0.776795          0.508189   
                   2             0.639566          0.483825   
                   3             0.724120          0.596222   
                   4             0.652073          0.572342   
OmniScaleCNN       1             0.774145          0.529399   
                   2             0.760804          0.517471   
                   3             0.797561          0.568172   
                   4             0.783862          0.555432   
StarRNN            1             0.788947          0.577289   
                   2             0.766099          0.548848   
                   3             0.781336          0.561198   
                   4             0.821071          0.566208   
TempCNN            1             0.791419          0.551983   
                   2             0.750792          0.530988   
                   3             0.779630          0.542562   
                   4             0.811772          0.559773   
TransformerEncoder 1             0.798653          0.585807   
                   2             0.788186          0.563306   
                   3             0.798976          0.568293   
                   4             0.816385          0.575565   

                         class-mean f-score  weighted f-score  kappa-metric  
model              fold                                                      
InceptionTime      1               0.550248          0.782731      0.725212  
                   2               0.510457          0.732667      0.684464  
                   3               0.477964          0.624437      0.549020  
                   4               0.448783          0.696238      0.617201  
LSTM               1               0.580049          0.788380      0.740102  
                   2               0.556520          0.765028      0.709150  
                   3               0.522199          0.722079      0.664367  
                   4               0.580665          0.820059      0.771737  
MSResNet           1               0.521983          0.761183      0.704569  
                   2               0.503944          0.623503      0.561879  
                   3               0.605147          0.710620      0.671051  
                   4               0.553298          0.664876      0.599489  
OmniScaleCNN       1               0.532591          0.740689      0.698362  
                   2               0.533298          0.757770      0.683162  
                   3               0.568137          0.792244      0.736579  
                   4               0.557601          0.787551      0.726165  
StarRNN            1               0.563707          0.774723      0.723026  
                   2               0.523391          0.701939      0.687076  
                   3               0.566424          0.770188      0.714481  
                   4               0.570891          0.804137      0.769026  
TempCNN            1               0.556241          0.783723      0.726197  
                   2               0.491144          0.711429      0.671149  
                   3               0.548378          0.777282      0.713442  
                   4               0.559938          0.804506      0.758891  
TransformerEncoder 1               0.582590          0.794793      0.737664  
                   2        

### Compile L2A Results

In [8]:
l2tables = []
for fold in [1,2,3,4]:
    table = load_table(f"{logdir}/L2A/{fold}").T
    table["fold"] = fold
    l2tables.append(table)

In [11]:

meantable = pd.concat(l2tables).groupby(["model"]).mean()
stdtable = pd.concat(l2tables).groupby(["model"]).std()

metrics = ["overall accuracy","average accuracy","class-mean f-score","weighted f-score","kappa-metric"]
models = ["OmniScaleCNN","TempCNN","MSResNet", "InceptionTime", "LSTM", "StarRNN", "TransformerEncoder"]
for metric in metrics:
    entries = list()
        
    for model in models:
        entries.append("$" + f"{meantable.loc[model,metric]:.2f}"+"^{\\pm "+ f"{stdtable.loc[model,metric]:.2f}" + "}$")
    row = f"{metric} & & " + " & ".join(entries)
    row += " \\\ "
    print(row)

overall accuracy & & $0.75^{\pm 0.06}$ & $0.79^{\pm 0.03}$ & $0.76^{\pm 0.02}$ & $0.73^{\pm 0.03}$ & $0.78^{\pm 0.05}$ & $0.78^{\pm 0.04}$ & $0.79^{\pm 0.03}$ \\ 
average accuracy & & $0.54^{\pm 0.04}$ & $0.55^{\pm 0.04}$ & $0.55^{\pm 0.04}$ & $0.51^{\pm 0.02}$ & $0.56^{\pm 0.03}$ & $0.55^{\pm 0.02}$ & $0.58^{\pm 0.03}$ \\ 
class-mean f-score & & $0.54^{\pm 0.04}$ & $0.55^{\pm 0.03}$ & $0.57^{\pm 0.04}$ & $0.49^{\pm 0.03}$ & $0.56^{\pm 0.03}$ & $0.56^{\pm 0.02}$ & $0.58^{\pm 0.02}$ \\ 
weighted f-score & & $0.73^{\pm 0.06}$ & $0.78^{\pm 0.04}$ & $0.76^{\pm 0.02}$ & $0.70^{\pm 0.06}$ & $0.78^{\pm 0.05}$ & $0.78^{\pm 0.04}$ & $0.79^{\pm 0.04}$ \\ 
kappa-metric & & $0.68^{\pm 0.08}$ & $0.72^{\pm 0.05}$ & $0.70^{\pm 0.03}$ & $0.65^{\pm 0.04}$ & $0.72^{\pm 0.06}$ & $0.72^{\pm 0.05}$ & $0.73^{\pm 0.05}$ \\ 


In [13]:
pd.concat(l2tables).groupby(["model","fold"]).first()

overall accuracy  average accuracy  \
model              fold                                       
InceptionTime      1             0.771050          0.533800   
                   2             0.730403          0.476804   
                   3             0.693775          0.523857   
                   4             0.734605          0.507277   
LSTM               1             0.800983          0.566342   
                   2             0.722138          0.517356   
                   3             0.781208          0.562395   
                   4             0.829336          0.580922   
MSResNet           1             0.766837          0.536133   
                   2             0.742245          0.508646   
                   3             0.786983          0.572476   
                   4             0.750512          0.598012   
OmniScaleCNN       1             0.734671          0.519379   
                   2             0.681896          0.500232   
                   3             0.760573          0.556235   
                   4             0.825972          0.583364   
StarRNN            1             0.789313          0.555245   
                   2             0.734545          0.515473   
                   3             0.797581          0.563910   
                   4             0.817591          0.560113   
TempCNN            1             0.794830          0.554077   
                   2             0.734118          0.497686   
                   3             0.803199          0.571277   
                   4             0.809609          0.575472   
TransformerEncoder 1             0.804552          0.584775   
                   2             0.743519          0.541460   
                   3             0.802638          0.582379   
                   4             0.823957          0.606289   

                         class-mean f-score  weighted f-score  kappa-metric  
model              fold                                                      
InceptionTime      1               0.536306          0.766904      0.703828  
                   2               0.502645          0.717395      0.643124  
                   3               0.462870          0.614834      0.603513  
                   4               0.471220          0.701496      0.659076  
LSTM               1               0.576579          0.799199      0.739637  
                   2               0.527994          0.720954      0.639460  
                   3               0.560205          0.764094      0.714349  
                   4               0.587238          0.827328      0.782179  
MSResNet           1               0.549065          0.769380      0.697171  
                   2               0.531518          0.737054      0.660065  
                   3               0.570622          0.769226      0.722196  
                   4               0.621109          0.746279      0.704581  
OmniScaleCNN       1               0.513839          0.716275      0.649812  
                   2               0.497293          0.670303      0.592605  
                   3               0.550792          0.726619      0.687063  
                   4               0.593317          0.819853      0.776827  
StarRNN            1               0.561890          0.789067      0.727189  
                   2               0.521541          0.732364      0.649706  
                   3               0.568356          0.792801      0.736649  
                   4               0.573021          0.815582      0.766583  
TempCNN            1               0.561041          0.787383      0.730600  
                   2               0.507293          0.728791      0.650568  
                   3               0.573574          0.799311      0.743531  
                   4               0.573071          0.810186      0.759289  
TransformerEncoder 1               0.590879          0.803417      0.745349  
                   2        